In [ ]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def pull_sql_NALC_GBA_equity(date):
    #Function that connects to our local SQL server and pulls Equity from NALC and GBA which are not in the data set.
    import pyodbc
    cn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "REDACTED;" #Server
                      "Database=Datawarehouse;"
                      "Trusted_Connection=yes;")
    df_sql = pd.read_sql_query(f"""
                select PortfolioDate, sum(MarketValueDKK/1000000) as 'Egenkapital'
                from Datawarehouse.Holdings.DailyFundTotal
                where PortfolioID in ('231201', '231101')
                and PortfolioDate = {date}
                Group by PortfolioDate
                Order by PortfolioDate DESC
                """,cn)
    df_sql = df_sql.reset_index(drop=True)

    return df_sql

def convert_datestring_to_date(string):
    string_m = string.split('-')[0][0:3]
    date_y = string.split('-')[1][0:4]
    date_m = 0
    date_d = '31'
    if string_m == 'jan':
        date_m = '01'
    elif string_m == 'feb':
        date_m = '02'
        date_d = '28'
        #Account for leap years
        if int(date_y)%4 == 0:
            date_d = '29'
            if int(date_y)%100 == 0:
                date_d = '28'
                if int(date_y)%400 == 0:
                    date_d = '29'
    elif string_m == 'mar':
        date_m = '03'
    elif string_m == 'apr':
        date_m = '04'
        date_d = '30'
    elif string_m == 'maj':
        date_m = '05'
    elif string_m == 'jun':
        date_m = '06'
        date_d = '30'
    elif string_m == 'jul':
        date_m = '07'
    elif string_m == 'aug':
        date_m = '08'
    elif string_m == 'sep':
        date_m = '09'
        date_d = '30'
    elif string_m == 'okt':
        date_m = '10'
    elif string_m == 'nov':
        date_m = '11'
        date_d = '30'
    else:
        date_m = '12'
    
    return f'{date_y}-{date_m}-{date_d}'

def pull_info_excel(file):
    sheet_names = ['2.1 Formue (A)', '2.1  Foreninger formue']
    try:
        df_formue = pd.read_excel(file,sheet_name=sheet_names[0],skiprows=2,skipfooter=3,usecols = [0, len(pd.read_excel(file,sheet_name=sheet_names[0],skiprows=2,skipfooter=3).columns)-1])
    except:
        df_formue = pd.read_excel(file,sheet_name=sheet_names[1],skiprows=2,skipfooter=3,usecols = [0, len(pd.read_excel(file,sheet_name=sheet_names[1],skiprows=2,skipfooter=3).columns)-1])
    month = f'{df_formue.columns[1][:-2]}-{file[-9:-5]}'
    fonds_navne = list(df_formue[df_formue.columns[0]][:-1])
    nettoformue = list(df_formue[df_formue.columns[1]][:-1])
    try:
        nalc_gba_egenkapital = pull_sql_NALC_GBA_equity(f"'{convert_datestring_to_date(month)}'").Egenkapital[0]
        formue_i_alt = list(df_formue[df_formue.columns[1]][-1:])[0] + nalc_gba_egenkapital
    except:
        formue_i_alt = list(df_formue[df_formue.columns[1]][-1:])[0]
        nalc_gba_egenkapital = 0
    market_share = []

    for i, formue in enumerate(nettoformue):
        #Sætter foreningsgrupper der har udenlandske fonde til marketshare = 0.0, da disse ikke indgår i totalen for danske foreninger
        string_navn = fonds_navne[i]
        #Manually add NALC and GBA funds' equity to net assets
        if string_navn == 'Kapitalforeningen Investering & Tryghed':
            market_share.append((formue+nalc_gba_egenkapital)/formue_i_alt)
            nettoformue[i] = formue+nalc_gba_egenkapital
            continue
        if string_navn.split('**')[0] != string_navn:
            market_share.append(0.0)
            continue
        #If value is 'nan' (this check works for some reason)
        elif formue != formue:
            market_share.append(0.0)
            continue
        market_share.append(formue/formue_i_alt)

    return fonds_navne, nettoformue, market_share, formue_i_alt, month

def supermerger(dictionary):
    #Takes input: Dictionary
    #   result[month] = fonds_navne, nettoformue, market_share, formue_i_alt
    list_month_dateformat = []
    list_iogt_nettoformue = []
    list_iogt_marketshare = []
    list_total_nettoformue = []
    
    list_fonde = []
    list_fonde_marketshare_thisyear, list_fonde_marketshare_lastyear = [], []
    
    count = 0
    for year in ['2022', '2021', '2020', '2019']:
        for month in ['december', 'november', 'oktober', 'september', 'august', 'juli', 'juni', 'maj', 'april', 'marts', 'februar', 'januar']:
            date = f'{month}-{year}'
            if count == 0:
                try:
                    global date_thisyear, date_lastyear #Global variables for latest marketshare YOY changes
                    date_thisyear = date
                    date_lastyear = f'{date[:-1]}{int(date[-1:])-1}' #Creates variable of date 12 months prior, e.g. november-2022 -> november-2021
                    for i, fond in enumerate(result[date][0]):
                        list_fonde.append(fond)
                        list_fonde_marketshare_thisyear.append(result[date][2][i])
                        try:
                            list_fonde_marketshare_lastyear.append(result[date_lastyear][2][result[date_lastyear][0].index(fond)])
                        except:
                            list_fonde_marketshare_lastyear.append(0.0) #If the fund didn't exist 12 months prior, we append marketshare 0.0 instead
                    count += 1
                except:
                    continue
            try:
                iogt_index = dictionary[date][0].index('Kapitalforeningen Investering & Tryghed')
                list_iogt_nettoformue.append(dictionary[date][1][iogt_index])
                list_iogt_marketshare.append(dictionary[date][2][iogt_index])
                list_total_nettoformue.append(dictionary[date][3])
                list_month_dateformat.append(convert_datestring_to_date(date))
            except:
                continue
    return list_month_dateformat, list_iogt_nettoformue, list_iogt_marketshare, list_total_nettoformue, list_fonde, list_fonde_marketshare_thisyear, list_fonde_marketshare_lastyear

path1 = r'REDACTED'
os.chdir(path1)

path_files = []
for file in os.listdir():
    try:
        if file.split(".")[1] == "xlsx":
            path_files.append(file)
    except:
        continue

result = {}
length_path_files = len(path_files)
counter = length_path_files//5
for i, file in enumerate(path_files):
    if i in [counter,counter*2,counter*3,counter*4,counter*5]:
        print(f'Pulled {i} of {length_path_files}')
    fonds_navne, nettoformue, market_share, formue_i_alt, month = pull_info_excel(file)
    result[month] = fonds_navne, nettoformue, market_share, formue_i_alt

date_thisyear, date_lastyear = 0, 0 #Global variables for latest marketshare YOY changes

dates, col1_1, col1_2, col1_3, col2_1, col2_2, col2_3  = supermerger(result)
dates.reverse(), col1_1.reverse(), col1_2.reverse(), col1_3.reverse()
df_allmonths = pd.DataFrame(list(zip(col1_1, col1_2, col1_3)), columns = ["I&T Nettoformue", "I&T Marketshare", "Nettoformue i alt"], index = dates)
df_changeYOY = pd.DataFrame(list(zip(col2_2, col2_3)), columns = [convert_datestring_to_date(date_thisyear), convert_datestring_to_date(date_lastyear)], index = col2_1)

#Gets dates of quarters and selects dataframe indices with these dates
quarters = []
for i, ind in enumerate(list(df_allmonths.index)):
    if ('-03-' in ind) or ('-06-' in ind) or ('-09-' in ind) or ('-12-' in ind):
        quarters.append(i)
df_quarters = df_allmonths.iloc[quarters]

#Write to Excel file
path_result = r'REDACTED'
os.chdir(path_result)

filename = 'Markedsstatistik til bestyrelsen.xlsx'

with pd.ExcelWriter(filename) as writer:
    df_allmonths.to_excel(writer, index = True, header = True, sheet_name = "I-T marketshare")
    df_quarters.to_excel(writer, index = True, header = True, sheet_name = "I-T marketshare kvartaler")
    df_changeYOY.to_excel(writer, index = True, header = True, sheet_name = "Alle fonde")

print("Finished.")

Pulled 8 of 40
Pulled 16 of 40
Pulled 24 of 40
Pulled 32 of 40
Finished.
